In [ ]:
## Concreteness

In [1]:
import csv
import glob
import pickle
import re
import xmltodict
import numpy as np
import random
import numpy as np
import pandas as pd

from datetime import datetime
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import xml.etree.ElementTree as ET
import xmltodict
import json

## Load the data

In [6]:
dt = pd.read_table("Concreteness_ratings_Brysbaert_et_al_BRM.txt")
dt

,Word,Bigram,Conc.M,Conc.SD,Unknown,Total,Percent_known,SUBTLEX,Dom_Pos
0,roadsweeper,0,4.85,0.37,1,27,0.96,0,0
1,traindriver,0,4.54,0.71,3,29,0.90,0,0
2,tush,0,4.45,1.01,3,25,0.88,66,0
3,hairdress,0,3.93,1.28,0,29,1.00,1,0
4,pharmaceutics,0,3.77,1.41,4,26,0.85,0,0
...,...,...,...,...,...,...,...,...,...
39949,unenvied,0,1.21,0.62,1,30,0.97,0,NaN
39950,agnostically,0,1.20,0.50,2,27,0.93,0,NaN
39951,conceptualistic,0,1.18,0.50,4,26,0.85,0,NaN
39952,conventionalism,0,1.18,0.48,1,29,0.97,0,NaN


We are considering the column `Conc.M`. The concreteness of a words are measured 1-5 where 5 represent where concrete and 1 astract. These ratings are more explained here: http://crr.ugent.be/papers/Brysbaert_Warriner_Kuperman_BRM_Concreteness_ratings.pdf s 

In [18]:
# Select the columns needed
data = dt[['Word', 'Conc.M']]
data.head(40)

,Word,Conc.M
0,roadsweeper,4.85
1,traindriver,4.54
2,tush,4.45
3,hairdress,3.93
4,pharmaceutics,3.77
5,hoover,3.76
6,shopkeeping,3.18
7,pushiness,2.48
8,underdevelop,2.37
9,tirelessness,2.28


In [22]:
data.shape

(39954, 2)

Explore the data

In [15]:
data['Conc.M'].describe()

count    39954.000000
mean         3.036267
std          1.038602
min          1.040000
25%          2.120000
50%          2.880000
75%          3.890000
max          5.000000
Name: Conc.M, dtype: float64

We need to decide whether a word is concrete or not. To determine the threshold. Every word which is 3> are considered concrete. This threshold is considered out of no rule, just because concreteness is given priority since the dataset was about concreteness

In [26]:
data['label'] = ['abstract' if x <=3 else 'concrete' for x in data['Conc.M']]

/Users/assoumerredempta/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [27]:
data

,Word,Conc.M,label
0,roadsweeper,4.85,concrete
1,traindriver,4.54,concrete
2,tush,4.45,concrete
3,hairdress,3.93,concrete
4,pharmaceutics,3.77,concrete
...,...,...,...
39949,unenvied,1.21,abstract
39950,agnostically,1.20,abstract
39951,conceptualistic,1.18,abstract
39952,conventionalism,1.18,abstract


In [32]:
data['Conc.M'][data['Conc.M'] <= 3].count()

21577

In [33]:
data['Conc.M'][data['Conc.M'] == 3].count()

399

In [35]:
data.rename(columns = {'Word':'word', 'Conc.M':'concreteness'}, inplace = True)
data.head(20)

/Users/assoumerredempta/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,word,concreteness,label
0,roadsweeper,4.85,concrete
1,traindriver,4.54,concrete
2,tush,4.45,concrete
3,hairdress,3.93,concrete
4,pharmaceutics,3.77,concrete
5,hoover,3.76,concrete
6,shopkeeping,3.18,concrete
7,pushiness,2.48,abstract
8,underdevelop,2.37,abstract
9,tirelessness,2.28,abstract


In [39]:
# data.to_csv('word_concreteness.csv')
# Save the dataset
# pickle.dump(casttrecSQS, open( "../DataSets/SQS/casttrecSQS.p", "wb" ))
pickle.dump(data, open( "../../FeatureExtraction/DataSets/word_concreteness.p", "wb" ))


In [36]:
pwd

'/Users/assoumerredempta/Documents/aSpring_2023/RYSe_Final/Data/DataSources'